<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-06-29 10:57:48


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [4]:
# set start and end date
start_date = (datetime.date.today() + datetime.timedelta(days=-5*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')



In [5]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


array(['5PAISA', 'AAVAS', 'AMARAJABAT', 'AMBUJACEM', 'ASIANPAINT',
       'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BATAINDIA', 'BAYERCROP',
       'CLEAN', 'DMART', 'EMAMILTD', 'FINEORG', 'GICRE', 'GILLETTE',
       'GLAND', 'HDFCAMC', 'ICICIGI', 'INDIGOPNTS', 'INDUSINDBK', 'INFY',
       'IRCTC', 'ITBEES', 'JMFINANCIL', 'JUBLFOOD', 'LTIM', 'LUXIND',
       'METROPOLIS', 'NIACL', 'PAGEIND', 'PIDILITIND', 'PNB',
       'RAJESHEXPO', 'RELAXO', 'RELIANCE', 'ROSSARI', 'SUNTV', 'TCS',
       'TEAMLEASE', 'UJJIVANSFB', 'VAIBHAVGBL', 'VALIANTORG', 'WIPRO'],
      dtype=object)

In [6]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Stock'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [7]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [8]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Max'] * tmp_df['Shares']
tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Juice Left%'] = tmp_df['Estimated P/L%'] - tmp_df['Current P/L%']

investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
previous = round(sum(tmp_df['Prev_Close']*tmp_df['Shares']),0)

today_pnl_amount = current-previous
today_pnl_percentage = round((current-previous)*100/previous,2)

curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['Weightage%'],0)

print('Investment: ', investment)
print('Current: ', current)
print('Today PnL: ', today_pnl_amount)
print('Today PnL %: ', today_pnl_percentage)
print('Current PnL: ', curr_pnl_amount)
print('Current PnL %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate PnL: ', est_pnl_amount)
print('Estimate PnL %: ', est_pnl_percentage)

qualified stocks: 44
Investment:  2944245.0
Current:  3040305.0
Today PnL:  12302.0
Today PnL %:  0.41
Current PnL:  96060.0
Current PnL %:  3.26
Estimate:  5273033.0
Estimate PnL:  2328788.0
Estimate PnL %:  79.1


In [9]:
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
18,1329.60,ICICIGI.NS,1613.0,1328.35,1137.80,69.0,V40,ATH,78508.20,91742.40,91656.15,111297.0,0.09,16.86,41.76,24.90,2.67,45.0
15,5104.00,GILLETTE.NS,7529.0,5081.40,4475.85,21.0,V40,ATH,93992.85,107184.00,106709.40,158109.0,0.44,14.03,68.21,54.18,3.19,45.0
1,1539.50,AAVAS.NS,3196.0,1508.75,1390.60,23.0,2T3Y,2T3Y,31983.80,35408.50,34701.25,73508.0,2.04,10.71,129.83,119.12,1.09,12.0
8,1669.50,BATAINDIA.NS,2173.0,1662.85,1523.62,100.0,V40,ATH,152362.00,166950.00,166285.00,217300.0,0.40,9.57,42.62,33.05,5.17,49.0
40,38.15,UJJIVANSFB.NS,57.0,37.95,34.87,1350.0,V40N,ATH,47074.50,51502.50,51232.50,76950.0,0.53,9.41,63.46,54.05,1.60,15.0
17,2041.50,HDFCAMC.NS,3518.0,2052.90,1879.66,41.0,V40,ATH,77066.06,83701.50,84168.90,144238.0,-0.56,8.61,87.16,78.55,2.62,23.0
11,3984.00,DMART.NS,5324.0,3872.10,3671.87,21.0,SS,ATH,77109.27,83664.00,81314.10,111804.0,2.89,8.50,44.99,36.49,2.62,22.0
39,2471.00,TEAMLEASE.NS,5281.0,2396.85,2279.93,21.0,V40N,2T3Y,47878.53,51891.00,50333.85,110901.0,3.09,8.38,131.63,123.25,1.63,14.0
4,3350.00,ASIANPAINT.NS,3542.0,3326.00,3091.34,25.0,V40,ATH,77283.50,83750.00,83150.00,88550.0,0.72,8.37,14.58,6.21,2.62,22.0
6,7069.65,BAJFINANCE.NS,7867.0,6989.25,6602.15,24.0,V40,ATH,158451.60,169671.60,167742.00,188808.0,1.15,7.08,19.16,12.08,5.38,38.0


In [10]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
16,1060.00,GLAND.NS,4316.0,1014.95,1225.87,55.0,2T3Y,2T3Y,67422.85,58300.00,55822.25,237380.0,4.44,-13.53,252.08,265.61,2.29,-31.0
42,557.00,VALIANTORG.NS,1851.0,547.35,542.52,144.0,2T3Y,2T3Y,78122.88,80208.00,78818.40,266544.0,1.76,2.67,241.19,238.52,2.65,7.0
27,1498.00,LUXIND.NS,4546.0,1506.75,1493.72,51.0,V40N,2T3Y,76179.72,76398.00,76844.25,231846.0,-0.58,0.29,204.34,204.05,2.59,1.0
41,309.25,VAIBHAVGBL.NS,966.0,309.50,317.48,144.0,2T3Y,2T3Y,45717.12,44532.00,44568.00,139104.0,-0.08,-2.59,204.27,206.86,1.55,-4.0
7,237.30,BANDHANBNK.NS,723.0,238.00,253.98,300.0,2T3Y,2T3Y,76194.00,71190.00,71400.00,216900.0,-0.29,-6.57,184.67,191.24,2.59,-17.0
29,117.20,NIACL.NS,311.0,118.30,117.07,390.0,TBD,ATH,45657.30,45708.00,46137.00,121290.0,-0.93,0.11,165.65,165.54,1.55,0.0
28,1458.80,METROPOLIS.NS,3440.0,1505.15,1365.98,22.0,NaN,ATH,30051.56,32093.60,33113.30,75680.0,-3.08,6.80,151.83,145.03,1.02,7.0
39,2471.00,TEAMLEASE.NS,5281.0,2396.85,2279.93,21.0,V40N,2T3Y,47878.53,51891.00,50333.85,110901.0,3.09,8.38,131.63,123.25,1.63,14.0
1,1539.50,AAVAS.NS,3196.0,1508.75,1390.60,23.0,2T3Y,2T3Y,31983.80,35408.50,34701.25,73508.0,2.04,10.71,129.83,119.12,1.09,12.0
19,1425.00,INDIGOPNTS.NS,3112.0,1423.30,1407.73,124.0,V40N,2T3Y,174558.52,176700.00,176489.20,385888.0,0.12,1.23,121.07,119.84,5.93,7.0


In [11]:
tmp_df.sort_values(by = 'Juice Left%', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
4,3350.00,ASIANPAINT.NS,3542.0,3326.00,3091.34,25.0,V40,ATH,77283.50,83750.00,83150.00,88550.0,0.72,8.37,14.58,6.21,2.62,22.0
35,2530.35,RELIANCE.NS,2811.0,2496.45,2467.15,31.0,V40,ATH,76481.65,78440.85,77389.95,87141.0,1.36,2.56,13.94,11.38,2.60,7.0
31,2603.00,PIDILITIND.NS,2902.0,2621.35,2504.06,30.0,V40,ATH,75121.80,78090.00,78640.50,87060.0,-0.70,3.95,15.89,11.94,2.55,10.0
6,7069.65,BAJFINANCE.NS,7867.0,6989.25,6602.15,24.0,V40,ATH,158451.60,169671.60,167742.00,188808.0,1.15,7.08,19.16,12.08,5.38,38.0
38,3221.00,TCS.NS,3857.0,3197.35,3231.00,16.0,V40,ATH,51696.00,51536.00,51157.60,61712.0,0.74,-0.31,19.37,19.68,1.76,-1.0
18,1329.60,ICICIGI.NS,1613.0,1328.35,1137.80,69.0,V40,ATH,78508.20,91742.40,91656.15,111297.0,0.09,16.86,41.76,24.90,2.67,45.0
5,1519.85,BAJAJFINSV.NS,1906.0,1522.15,1454.93,52.0,V40,ATH,75656.36,79032.20,79151.80,99112.0,-0.15,4.46,31.00,26.54,2.57,11.0
23,30.27,ITBEES.NS,40.0,30.13,30.07,1675.0,SS,ATH,50367.25,50702.25,50467.75,67000.0,0.46,0.67,33.02,32.35,1.71,1.0
8,1669.50,BATAINDIA.NS,2173.0,1662.85,1523.62,100.0,V40,ATH,152362.00,166950.00,166285.00,217300.0,0.40,9.57,42.62,33.05,5.17,49.0
3,435.95,AMBUJACEM.NS,588.0,433.85,424.02,180.0,TBD,ATH,76323.60,78471.00,78093.00,105840.0,0.48,2.81,38.67,35.86,2.59,7.0


In [12]:
tmp_df.sort_values(by = 'Weightage%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
19,1425.00,INDIGOPNTS.NS,3112.0,1423.30,1407.73,124.0,V40N,2T3Y,174558.52,176700.00,176489.20,385888.0,0.12,1.23,121.07,119.84,5.93,7.0
6,7069.65,BAJFINANCE.NS,7867.0,6989.25,6602.15,24.0,V40,ATH,158451.60,169671.60,167742.00,188808.0,1.15,7.08,19.16,12.08,5.38,38.0
8,1669.50,BATAINDIA.NS,2173.0,1662.85,1523.62,100.0,V40,ATH,152362.00,166950.00,166285.00,217300.0,0.40,9.57,42.62,33.05,5.17,49.0
30,37499.00,PAGEIND.NS,53549.0,37916.45,39325.03,3.0,V40,ATH,117975.09,112497.00,113749.35,160647.0,-1.10,-4.64,36.17,40.81,4.01,-19.0
15,5104.00,GILLETTE.NS,7529.0,5081.40,4475.85,21.0,V40,ATH,93992.85,107184.00,106709.40,158109.0,0.44,14.03,68.21,54.18,3.19,45.0
0,359.25,5PAISA.NS,553.0,358.50,347.06,228.0,V40N,ATH,79129.68,81909.00,81738.00,126084.0,0.21,3.51,59.34,55.83,2.69,9.0
18,1329.60,ICICIGI.NS,1613.0,1328.35,1137.80,69.0,V40,ATH,78508.20,91742.40,91656.15,111297.0,0.09,16.86,41.76,24.90,2.67,45.0
42,557.00,VALIANTORG.NS,1851.0,547.35,542.52,144.0,2T3Y,2T3Y,78122.88,80208.00,78818.40,266544.0,1.76,2.67,241.19,238.52,2.65,7.0
25,495.00,JUBLFOOD.NS,903.0,494.20,471.56,164.0,SS,ATH,77335.84,81180.00,81048.80,148092.0,0.16,4.97,91.49,86.52,2.63,13.0
17,2041.50,HDFCAMC.NS,3518.0,2052.90,1879.66,41.0,V40,ATH,77066.06,83701.50,84168.90,144238.0,-0.56,8.61,87.16,78.55,2.62,23.0


In [13]:
tmp_df.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

In [14]:
tmp_df.sort_values(by = 'Risk Ind', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
16,1060.00,GLAND.NS,4316.0,1014.95,1225.87,55.0,2T3Y,2T3Y,67422.85,58300.00,55822.25,237380.0,4.44,-13.53,252.08,265.61,2.29,-31.0
30,37499.00,PAGEIND.NS,53549.0,37916.45,39325.03,3.0,V40,ATH,117975.09,112497.00,113749.35,160647.0,-1.10,-4.64,36.17,40.81,4.01,-19.0
7,237.30,BANDHANBNK.NS,723.0,238.00,253.98,300.0,2T3Y,2T3Y,76194.00,71190.00,71400.00,216900.0,-0.29,-6.57,184.67,191.24,2.59,-17.0
33,542.00,RAJESHEXPO.NS,949.0,544.40,624.78,48.0,V40N,ATH,29989.44,26016.00,26131.20,45552.0,-0.44,-13.25,51.89,65.14,1.02,-14.0
43,382.30,WIPRO.NS,713.0,382.60,388.07,195.0,2T3Y,2T3Y,75673.65,74548.50,74607.00,139035.0,-0.08,-1.49,83.73,85.22,2.57,-4.0
41,309.25,VAIBHAVGBL.NS,966.0,309.50,317.48,144.0,2T3Y,2T3Y,45717.12,44532.00,44568.00,139104.0,-0.08,-2.59,204.27,206.86,1.55,-4.0
37,435.35,SUNTV.NS,710.0,437.90,446.75,68.0,V40N,ATH,30379.00,29603.80,29777.20,48280.0,-0.58,-2.55,58.93,61.48,1.03,-3.0
12,412.00,EMAMILTD.NS,588.0,417.55,422.59,72.0,V200,NaN,30426.48,29664.00,30063.60,42336.0,-1.33,-2.51,39.14,41.65,1.03,-3.0
32,50.60,PNB.NS,95.0,50.85,50.85,900.0,SS,ATH,45765.00,45540.00,45765.00,85500.0,-0.49,-0.49,86.82,87.31,1.55,-1.0
38,3221.00,TCS.NS,3857.0,3197.35,3231.00,16.0,V40,ATH,51696.00,51536.00,51157.60,61712.0,0.74,-0.31,19.37,19.68,1.76,-1.0


In [15]:
print('Total profit: ',round(sum(tmp_df[tmp_df['Current P/L%'] > 0]['Current']) - sum(tmp_df[tmp_df['Current P/L%'] > 0]['Investment']),0))
print('Total loss: ',round(sum(tmp_df[tmp_df['Current P/L%'] < 0]['Current']) - sum(tmp_df[tmp_df['Current P/L%'] < 0]['Investment']),0))

Total profit:  124413.0
Total loss:  -28353.0


In [16]:
# SIP based on portfolio
tmp_df[tmp_df['Weightage%'] < 2].sort_values(by = 'Current P/L%', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
33,542.00,RAJESHEXPO.NS,949.0,544.40,624.78,48.0,V40N,ATH,29989.44,26016.00,26131.20,45552.0,-0.44,-13.25,51.89,65.14,1.02,-14.0
41,309.25,VAIBHAVGBL.NS,966.0,309.50,317.48,144.0,2T3Y,2T3Y,45717.12,44532.00,44568.00,139104.0,-0.08,-2.59,204.27,206.86,1.55,-4.0
37,435.35,SUNTV.NS,710.0,437.90,446.75,68.0,V40N,ATH,30379.00,29603.80,29777.20,48280.0,-0.58,-2.55,58.93,61.48,1.03,-3.0
12,412.00,EMAMILTD.NS,588.0,417.55,422.59,72.0,V200,NaN,30426.48,29664.00,30063.60,42336.0,-1.33,-2.51,39.14,41.65,1.03,-3.0
32,50.60,PNB.NS,95.0,50.85,50.85,900.0,SS,ATH,45765.00,45540.00,45765.00,85500.0,-0.49,-0.49,86.82,87.31,1.55,-1.0
10,1392.00,CLEAN.NS,2648.0,1386.85,1396.49,23.0,2T3Y,2T3Y,32119.27,32016.00,31897.55,60904.0,0.37,-0.32,89.62,89.94,1.09,-0.0
38,3221.00,TCS.NS,3857.0,3197.35,3231.00,16.0,V40,ATH,51696.00,51536.00,51157.60,61712.0,0.74,-0.31,19.37,19.68,1.76,-1.0
29,117.20,NIACL.NS,311.0,118.30,117.07,390.0,TBD,ATH,45657.30,45708.00,46137.00,121290.0,-0.93,0.11,165.65,165.54,1.55,0.0
23,30.27,ITBEES.NS,40.0,30.13,30.07,1675.0,SS,ATH,50367.25,50702.25,50467.75,67000.0,0.46,0.67,33.02,32.35,1.71,1.0
26,5107.00,LTIM.NS,7439.0,5163.05,4996.37,10.0,V200,NaN,49963.70,51070.00,51630.50,74390.0,-1.09,2.21,48.89,46.68,1.70,4.0
